<a href="https://colab.research.google.com/github/Mahmuda-104/Thesis/blob/main/CNN_New1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
from scipy.io import loadmat

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/My Drive/Data'

In [4]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')
for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []
for control_data_path in control_data:
    n_epochs = 30 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{control_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        # b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        # filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        filtered_data=mat_data1
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print( Control30[0].shape)


(30, 60, 1000)


In [5]:
print("Filtered and downsampled data shape:", Control30.shape)

Filtered and downsampled data shape: (32, 30, 60, 1000)


In [6]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed49 = []
for concussed_data_path in concussed_data:
    n_epochs = 30 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        # b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        # filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        filtered_data=mat_data1
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed49.append(np.array(concussed_arr))

Concussed49 = np.array(Concussed49)
print( Concussed49[0].shape)


(30, 60, 1000)


In [15]:
print("Filtered and downsampled data shape:", Concussed49.shape)

Filtered and downsampled data shape: (52, 30, 60, 1000)


In [8]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# dataset = np.concatenate((Control30, Concussed49), axis=0)

# #(0 for control, 1 for concussed)
# labels = np.concatenate((np.zeros(len(Control30)), np.ones(len(Concussed49))))


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Define your dataset and labels
dataset = np.concatenate((Control30, Concussed49), axis=0)
labels = np.concatenate((np.zeros(len(Control30)), np.ones(len(Concussed49))))

# Define the number of training datasets you want to use
num_train_datasets = 5

# Define the size of the training set and testing set
train_ratio = 0.8
test_ratio = 0.2

# Calculate the total number of datasets and the step size for each iteration
total_datasets = len(dataset)
step_size = int((total_datasets - train_ratio * total_datasets) / (num_train_datasets - 1))

for i in range(num_train_datasets):
    # Calculate the starting index for the current training set
    start_idx = i * step_size

    # Calculate the ending index for the current training set
    end_idx = int(start_idx + train_ratio * total_datasets)

    # Extract the current training set
    X_train = dataset[start_idx:end_idx]
    y_train = labels[start_idx:end_idx]

    # Extract the current testing set
    X_test = dataset[end_idx:]
    y_test = labels[end_idx:]

    # Normalize the input data (if required)
    # X_train = X_train / 255.0
    # X_test = X_test / 255.0

    # Reshape the input data based on the expected input shape of the model
    input_shape = X_train.shape[1:]
    X_train = X_train.reshape((-1,) + input_shape)
    X_test = X_test.reshape((-1,) + input_shape)

    # Build the model
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(16, kernel_size=(6, 6), strides=2, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))

    learning_rate = 0.005  # Specify your desired learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Evaluate the model on the testing set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)

    # Print the iteration number and test accuracy
    print("Iteration:", i+1)
    print("Test Accuracy:", test_accuracy)


1/1 [==============================] - 1s 1s/step - loss: 1.7850 - accuracy: 0.8824
Iteration: 1
Test Accuracy: 0.8823529481887817
1/1 [==============================] - 1s 873ms/step - loss: 0.6240 - accuracy: 1.0000
Iteration: 2
Test Accuracy: 1.0
1/1 [==============================] - 0s 454ms/step - loss: 0.4565 - accuracy: 1.0000
Iteration: 3
Test Accuracy: 1.0
1/1 [==============================] - 0s 317ms/step - loss: 0.4287 - accuracy: 1.0000
Iteration: 4
Test Accuracy: 1.0


1/1 [==============================] - 0s 194ms/step - loss: 0.3097 - accuracy: 1.0000
Iteration: 5
Test Accuracy: 1.0


In [13]:
print("Test Accuracy:", test_accuracy)

NameError: ignored

In [10]:
train_ratio = 0.8  # 80% for training
val_ratio = 0.1   # 10% for validation
test_ratio = 0.1  # 10% for testing

In [11]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

NameError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

# Normalize the input data (if required)
# X_train = X_train / 255.0
# X_val = X_val / 255.0
# X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6), strides=2, activation='relu', padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

learning_rate = 0.005  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
val_loss, val_accuracy = model.evaluate(X_val, y_val)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Define a function that creates your CNN model
def create_model(learning_rate=0.005, dropout_rate=0.5):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(16, kernel_size=(6, 6), strides=2, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(2, activation='softmax'))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier based on your model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters and their possible values for grid search
param_grid = {
    'learning_rate': [0.001, 0.005, 0.01],
    'dropout_rate': [0.3, 0.5, 0.7],
    'epochs': [10, 20, 30],
    'batch_size': [16, 32, 64]
}

# Create the grid search object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.model.evaluate(X_test, y_test)
val_accuracy = best_model.model.evaluate(X_val, y_val)

# Print the best parameters and test accuracy
print("Best Parameters:", best_params)
print("Test Accuracy:", test_accuracy)
print("Validation Accuracy:", val_accuracy)

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy (using predictions):", test_accuracy)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# Normalize the input data (if required)
# X_train = X_train / 255.0
# X_val = X_val / 255.0
# X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6),strides=2, activation='relu',padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with the desired loss function and optimizer
learning_rate = 0.05  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

# # Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# # Normalize the input data (if required)
# # X_train = X_train / 255.0
# # X_val = X_val / 255.0
# # X_test = X_test / 255.0

# # Reshape the input data based on the expected input shape of the model
# input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
# X_train = X_train.reshape((-1,) + input_shape)
# X_val = X_val.reshape((-1,) + input_shape)
# X_test = X_test.reshape((-1,) + input_shape)

# # Build the CNN model
# model = tf.keras.Sequential()
# model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='linear'))

# # Compile the model with the desired loss function and optimizer
# learning_rate = 0.05  # Specify your desired learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)

# # Print the test accuracy
# print("Test Accuracy:", test_accuracy)
